In [11]:
from clova import HyperCLOVAChatModel
from langchain_core.messages import HumanMessage
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import json
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# 벡터 저장소 초기화
loader = TextLoader("/Users/kdb/Desktop/stockelper/hyperclova/context.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)

# HyperCLOVA 모델 초기화
chat_model = HyperCLOVAChatModel(
    temperature=0.7,
    max_tokens=256,
    top_p=0.8,
    top_k=0,
    repeat_penalty=5.0
)

# 테스트용 질문
query = "여기에 테스트 질문을 입력하세요"

# 관련 문서 검색
relevant_docs = vectorstore.similarity_search(query)
context = "\n".join([doc.page_content for doc in relevant_docs])

# 프롬프트 구성
prompt = f"""다음 컨텍스트를 바탕으로 질문에 답변해주세요:

컨텍스트:
{context}

질문: {query}
답변:"""

messages = [HumanMessage(content=prompt)]

# 스트리밍 응답 출력
for chunk in chat_model.stream(messages):
    if "data:" in chunk.content:
        try:
            data_str = chunk.content.split("data:")[1].strip()
            data = json.loads(data_str)
            if 'message' in data and 'content' in data['message']:
                content = data['message']['content']
                if content:
                    print(content, end='', flush=True)
        except json.JSONDecodeError:
            continue
print()  # 줄바꿈

테스트 질문이 없어서 답변을 드릴 수 없습니다. 어떤 주식 관련 질문이든지 제가 알고 있는 지식과 경험을 바탕으로 정확한 답변을 드리겠습니다. 예를 들어, "어떤 주식 종목을 추천하시나요?" 또는 "주식 시장 동향은 어떻게 되나요?"와 같은 질문도 가능합니다. 언제든지 물어보세요!테스트 질문이 없어서 답변을 드릴 수 없습니다. 어떤 주식 관련 질문이든지 제가 알고 있는 지식과 경험을 바탕으로 정확한 답변을 드리겠습니다. 예를 들어, "어떤 주식 종목을 추천하시나요?" 또는 "주식 시장 동향은 어떻게 되나요?"와 같은 질문도 가능합니다. 언제든지 물어보세요!
